In [9]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import json
import sys
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

directory = "/kaggle/working/"
user_data = directory + "data/"
valid_data = directory + "data/"
test_data = directory + "label_book/" # this can be the label book, or any other test set you create

# Rules

- Submission must have less than 10,000 images combined in training and validation

**Submissions will be evaluated according to two categories:**
1. **Best Performance Overall**
2. **Most Innovative**

# Getting started


First, download the dataset by clicking into the bundle (https://worksheets.codalab.org/bundles/0xcea1d733e1f144d9aba83929af51f191) and press the download button. The dataset contains ~3000 images of handwritten roman numerals 1-10. **Your task is to optimize model performance by improving the dataset and making training and validation splits.**

Your submission should be a zip file with the following file structure, and with your training and validation data split into different folders. You can rename `sample_submission` whatever you like, but *the name of your zip file should match your folder name*. In this example, it should be called `sample_submission.zip`. When the zip file is expanded, the file structure and names inside must match the following:

    sample_submission/
        train/
            i/
            ii/
            iii/
            iv/
            ...
        val/
            i/
            ii/
            iii/
            iv/
            ...


You can try fixing incorrect labels, adding data for side case tuning, apply data augmentation techniques, or use any other method to improve the data. You may also find it helpful to take a look at the training script to get a better sense of the preprocessing and model (these are held fixed). The script will resize all images to `(32, 32)` and run them through a cut off ResNet50. 
[dataset train.py](https://worksheets.codalab.org/bundles/0x57030e4a2d034af4b8efa38df4ff6af6)

If you choose to create your own data, you may find this script helpful for converting your images:
[dataset convert.py](https://worksheets.codalab.org/bundles/0x3a2514052d9240ffa24774a7092b82b8)

We also provide you with a label book that has five examples for each label. You can use it as a sample test set as you create your submission: 
[dataset label_book](https://worksheets.codalab.org/bundles/0xe1d24af2ca5e4d95be653f1c7877125e)

In [2]:
!pip install --upgrade --no-cache-dir gdown

In [3]:
# https://drive.google.com/file/d/1z-kOIm8EWuxGW-hD_i6DrjYU2C3kzlst/view?usp=sharing
!gdown --id 1z-kOIm8EWuxGW-hD_i6DrjYU2C3kzlst

In [4]:
# https://drive.google.com/file/d/1DUD1Lh7f5RcTHihkylNuhy4ozRGtoA_z/view?usp=sharing
!gdown --id 1DUD1Lh7f5RcTHihkylNuhy4ozRGtoA_z

In [5]:
!tar zxf data.tar.gz
!tar zxf test.gz

In [6]:
!ls label_book

In [10]:
### DO NOT MODIFY BELOW THIS LINE, THIS IS THE FIXED MODEL ###
batch_size = 8
tf.random.set_seed(123)

train = tf.keras.preprocessing.image_dataset_from_directory(
        user_data + '/train',
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=True,
        seed=123,
        batch_size=batch_size,
        image_size=(32, 32),
    )

valid = tf.keras.preprocessing.image_dataset_from_directory(
        user_data + '/val',
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=True,
        seed=123,
        batch_size=batch_size,
        image_size=(32, 32),
)

total_length = ((train.cardinality() + valid.cardinality()) * batch_size).numpy()

#if total_length > 10_000:
#    print(f"Dataset size larger than 10,000. Got {total_length} examples")
#    sys.exit()

test = tf.keras.preprocessing.image_dataset_from_directory(
        test_data,
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=False,
        seed=123,
        batch_size=batch_size,
        image_size=(32, 32),
)

base_model = tf.keras.applications.ResNet50(
        input_shape=(32, 32, 3),
        include_top=False,
        weights=None,
)
base_model = tf.keras.Model(
        base_model.inputs, outputs=[base_model.get_layer("conv2_block3_out").output]
)

inputs = tf.keras.Input(shape=(32, 32, 3))
x = tf.keras.applications.resnet.preprocess_input(inputs)
x = base_model(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(10)(x)
model = tf.keras.Model(inputs, x)

model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=0.0001),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
)
model.summary()
    
loss_0, acc_0 = model.evaluate(valid)
print(f"loss {loss_0}, acc {acc_0}")

checkpoint = tf.keras.callbacks.ModelCheckpoint(
        "best_model",
        monitor="val_accuracy",
        mode="max",
        save_best_only=True,
        save_weights_only=True,
)

history = model.fit(
        train,
        validation_data=valid,
        epochs=100,
        callbacks=[checkpoint],
)

model.load_weights("best_model")

loss, acc = model.evaluate(valid)
print(f"final loss {loss}, final acc {acc}")

test_loss, test_acc = model.evaluate(test)
print(f"test loss {test_loss}, test acc {test_acc}")